In [ ]:
"""
FANTOM5データを活用した人工プロモーターライブラリ作成システム
Keiさんのオリジナルコード + 充実したFANTOM5/JASPARモチーフデータベース

【改良点】
- 実際のFANTOM5論文とJASPARデータベースに基づくモチーフを追加
- 組織特異性を強化（liver, brain, heart, immune, stem cell対応）
- コアプロモーター要素を分類・充実
- オリジナルの全機能を保持
"""

import pandas as pd
from itertools import product
import requests
from io import StringIO
import re

# ============================================================================
# 基本関数群（元のコードの修正版）
# ============================================================================

def Coplanar(core_promoter, TFB, spacer_length=10):
    """
    DNAの同じ側に必要なタンパク質が結合できるようにする
    """
    if not core_promoter or not TFB:
        raise ValueError('core promoterとTFBは空です')
    spacer = 'N' * spacer_length
    artificial_promoter = TFB + spacer + core_promoter
    return artificial_promoter


def Opposite(core_promoter, TFB, spacer_length=5):
    """
    DNAの反対側に必要なタンパク質が結合できるようにする
    """
    if not core_promoter or not TFB:
        raise ValueError('core promoterとTFBは空です')
    spacer = 'N' * spacer_length
    artificial_promoter = TFB + spacer + core_promoter
    return artificial_promoter


def design_promoter_with_custom_spacer(core_promoter, TFB, spacer_sequence):
    """具体的なspacer配列を指定する"""
    return TFB + spacer_sequence + core_promoter


def validate_sequence(sequence):
    """入力した配列がDNAかどうかを確認する関数"""
    valid_bases = set('ATGCNatgcnWSYRKMVHDBwsyrkmvhdb')
    return all(base in valid_bases for base in sequence)


def generate_promoter_library(core_csv=None, tfb_csv=None,
                              core_list=None, tfb_list=None,
                              promoter_types=['Coplanar', 'Opposite'],
                              coplanar_spacer=10, opposite_spacer=5,
                              output_csv=None):
    """
    プロモーターライブラリを生成する（修正版）
    """
    # Core promoterの読み込み
    if core_csv:
        core_df = pd.read_csv(core_csv)
        core_col = 'sequence' if 'sequence' in core_df.columns else core_df.columns[0]
        cores = core_df[core_col].tolist()
    elif core_list:
        cores = core_list
    else:
        raise ValueError('core_csvまたはcore_listを指定してください')
    
    # TFBの読み込み
    if tfb_csv:
        tfb_df = pd.read_csv(tfb_csv)
        tfb_col = 'sequence' if 'sequence' in tfb_df.columns else tfb_df.columns[0]
        tfbs = tfb_df[tfb_col].tolist()
    elif tfb_list:
        tfbs = tfb_list
    else:
        raise ValueError('tfb_csvまたはtfb_listを指定してください')

    # 全ての組み合わせを生成
    library = []
    for i, (core, tfb) in enumerate(product(cores, tfbs), 1):
        entry = {
            'ID': f'Promoter_{i:05d}',
            'Core_Promoter': core, 
            'TFB': tfb,
        }

        if 'Coplanar' in promoter_types:
            Coplanar_seq = Coplanar(core, tfb, spacer_length=coplanar_spacer)
            entry['Coplanar_sequence'] = Coplanar_seq
            entry['Coplanar_Length'] = len(Coplanar_seq)

        if 'Opposite' in promoter_types:
            Opposite_seq = Opposite(core, tfb, spacer_length=opposite_spacer)
            entry['Opposite_Sequence'] = Opposite_seq
            entry['Opposite_length'] = len(Opposite_seq)
        
        library.append(entry)

    # libraryをDataFrameに変換
    library_df = pd.DataFrame(library)

    # CSVにして出力する
    if output_csv:
        library_df.to_csv(output_csv, index=False)
        print(f'ライブラリを{output_csv}に保存しました')
    
    return library_df


# ============================================================================
# FANTOM5データ統合機能（モチーフデータベース強化版）
# ============================================================================

class FANTOM5DataHandler:
    """
    FANTOM5データを取得・処理するクラス
    
    【データソース】
    - FANTOM5論文：組織特異的転写因子の同定
    - JASPARデータベース：転写因子結合モチーフ
    - 文献：コアプロモーター要素
    """
    
    #まずはFANTOM5DataHandlerオブジェクトのインスタンス変数を定義。
    #これらは、オブジェクト.属性アクセスで、インスタンス変数へアクセス可能。
    def __init__(self):
        self.base_url = "https://fantom.gsc.riken.jp/5/"
        
        #FANTOM5DataHandlerオブジェクトに固有のデータを保持させる。
        #まずは組織特異的TFdatabaseのデータを保持させる。
        # ================================================================
        # 組織特異的転写因子データベース（FANTOM5論文から）
        # ================================================================
        self.tissue_specific_tfs = {
            'liver': [
                'HNF1A', 'HNF1B', 'HNF4A', 'HNF4G',      # 肝臓核因子
                'CEBPA', 'CEBPB', 'CEBPG',               # CCAAT/enhancer binding protein
                'FOXA1', 'FOXA2', 'FOXA3',               # Forkhead box A
                'ONECUT1', 'ONECUT2',                    # One cut homeobox
                'NR1H3', 'NR1H4',                        # Nuclear receptor (LXR)
            ],
            'brain': [
                'NEUROD1', 'NEUROD2', 'NEUROD6',         # Neurogenic differentiation
                'NEUROG1', 'NEUROG2',                    # Neurogenin
                'SOX2', 'SOX3', 'SOX11',                 # SRY-box transcription factor
                'REST', 'NRSF',                          # RE1-silencing transcription factor
                'CREB1', 'CREB3',                        # cAMP responsive element binding
                'MEF2A', 'MEF2C', 'MEF2D',               # Myocyte enhancer factor
                'TBR1', 'TBR2',                          # T-box brain transcription factor
            ],
            'heart': [
                'GATA4', 'GATA5', 'GATA6',               # GATA binding protein
                'MEF2A', 'MEF2C', 'MEF2D',               # Myocyte enhancer factor
                'NKX2-5', 'NKX2-6',                      # NK2 homeobox
                'TBX5', 'TBX20',                         # T-box transcription factor
                'SRF',                                   # Serum response factor
                'HAND1', 'HAND2',                        # Heart and neural crest derivatives
            ],
            'immune': [
                'NFKB1', 'NFKB2', 'RELA', 'RELB',        # NF-kappa-B
                'STAT1', 'STAT3', 'STAT4', 'STAT6',      # Signal transducer and activator
                'IRF1', 'IRF3', 'IRF7', 'IRF8',          # Interferon regulatory factor
                'SPI1', 'PU1',                           # PU.1 (Spi-1 proto-oncogene)
                'ETS1', 'ETS2',                          # ETS proto-oncogene
                'RUNX1', 'RUNX3',                        # Runt-related transcription factor
            ],
            'stem': [
                'POU5F1', 'OCT4',                        # Octamer-binding transcription factor
                'SOX2',                                  # SRY-box transcription factor 2
                'NANOG',                                 # Nanog homeobox
                'KLF4',                                  # Kruppel-like factor 4
                'MYC', 'CMYC',                           # MYC proto-oncogene
                'LIN28A', 'LIN28B',                      # Lin-28 homolog
            ],
            'muscle': [
                'MYOD1', 'MYOG',                         # Myogenic differentiation
                'MEF2A', 'MEF2C',                        # Myocyte enhancer factor
                'SRF',                                   # Serum response factor
                'PAX3', 'PAX7',                          # Paired box gene
            ],
        }
        #次に転写因子結合モチーフ一覧情報をclassにインスタンス変数として持たせる。
        # ================================================================
        # 転写因子結合モチーフ（JASPARデータベースから）
        # ================================================================
        self.transcription_factor_motifs = {
            # liver-specific TFs
            'HNF1A': 'GTTAATNATTAAC',
            'HNF1B': 'GTTAATGATTAAC',
            'HNF4A': 'AGGTCA',
            'HNF4G': 'RGGTCA',
            'CEBPA': 'TTGCGCAA',
            'CEBPB': 'ATTGCGCAAT',
            'CEBPG': 'TTGCGCAA',
            'FOXA1': 'TGTTTAC',
            'FOXA2': 'TGTTTGC',
            'FOXA3': 'TGTTTRY',
            'ONECUT1': 'ATCAAT',
            'ONECUT2': 'ATCAAT',
            'NR1H3': 'AGGTCA',
            'NR1H4': 'AGGTCA',
            
            # brain-specific TFs
            'NEUROD1': 'CAGCTG',
            'NEUROD2': 'CAGCTG',
            'NEUROD6': 'CAGSTG',
            'NEUROG1': 'CAGCTG',
            'NEUROG2': 'CAGSTG',
            'SOX2': 'CATTGTT',
            'SOX3': 'CATTGT',
            'SOX11': 'CATTGTT',
            'REST': 'NRSENSRENSTKAN',
            'NRSF': 'TTCAGCACCMTGMN',
            'CREB1': 'TGACGTCA',
            'CREB3': 'TGACGTCA',
            'MEF2A': 'CTAWWWWTAG',
            'MEF2C': 'CTAAAAATAG',
            'MEF2D': 'YTAWWWWTAR',
            'TBR1': 'TCACCT',
            'TBR2': 'TCACCT',
            
            # heart-specific TFs
            'GATA4': 'WGATAR',
            'GATA5': 'AGATAA',
            'GATA6': 'AGATAA',
            'NKX2-5': 'TNAAGTG',
            'NKX2-6': 'TNAAGTG',
            'TBX5': 'TCACACCT',
            'TBX20': 'TCACACCT',
            'SRF': 'CCWWWWWWGG',
            'HAND1': 'CANNTG',
            'HAND2': 'CANNTG',
            
            # immune cell-specific
            'NFKB1': 'GGGACTTTCC',
            'NFKB2': 'GGGRATTTCC',
            'RELA': 'GGGAMTTYCC',
            'RELB': 'GGGAMTTYCC',
            'STAT1': 'TTCNNNGAA',
            'STAT3': 'TTCNNNGAA',
            'STAT4': 'TTCYNRGAA',
            'STAT6': 'TTCYNRGAA',
            'IRF1': 'GAAANNGAAA',
            'IRF3': 'GAAAN',
            'IRF7': 'GAAANNGAAA',
            'IRF8': 'GAAANNGAAA',
            'SPI1': 'GGAAGT',
            'PU1': 'GAGGAA',
            'ETS1': 'GGAA',
            'ETS2': 'GGAA',
            'RUNX1': 'TGTGGT',
            'RUNX3': 'TGTGGT',
            
            # stem cell-specific TFs
            'POU5F1': 'ATGCAAAT',
            'OCT4': 'ATGCAAAT',
            'NANOG': 'TAATGG',
            'KLF4': 'CACCC',
            'MYC': 'CACGTG',
            'CMYC': 'CACGTG',
            'LIN28A': 'AAGNAG',
            'LIN28B': 'AAGNAG',
            
            # muscle-specific TFs
            'MYOD1': 'CANNTG',
            'MYOG': 'CANNTG',
            'PAX3': 'GTCACGC',
            'PAX7': 'GTCACGC',
            
            # ubiquitous TFs
            'SP1': 'GGGCGG',
            'SP3': 'GGGGCGGGG',
            'AP1': 'TGASTCA',
            'AP2': 'GCCNNNGGC',
            'YY1': 'CGCCATNTT',
            'NRF1': 'GCGCATGCGC',
            'E2F1': 'TTTCGCGC',
            'MYC_MAX': 'CACGTG',
        }
        
        #次にcore promoterのモチーフ情報をclassに保持させる。
        # ================================================================
        # コアプロモーター要素（文献から）
        # ================================================================
        self.core_promoter_elements = {
            # TATA box (約25%のプロモーターに存在)
            'TATA_box': [
                'TATAAA',      # コンセンサス配列
                'TATAWA',      # バリアント (W = A or T)
                'TATATAA',     # よく見られるバリアント
                'TATAAAA',     # 長いバージョン
                'CTATAA',      # 弱いTATA box
            ],
            
            # Initiator (Inr) 要素 (約50%のプロモーターに存在)
            'Initiator': [
                'YYANWYY',     # コンセンサス (Y=C/T, W=A/T, N=any)
                'YYANWYW',     # バリアント
                'CCANWTY',     # よく見られるパターン
                'TCAKTY',      # 別のバリアント (K=G/T)
            ],
            
            # Downstream Promoter Element (DPE)
            'DPE': [
                'RGWYV',       # コンセンサス (R=A/G, W=A/T, Y=C/T, V=A/C/G)
                'AGWYW',       # バリアント
                'AGACY',       # よく見られるパターン
            ],
            
            # TFIIB Recognition Element (BRE)
            'BRE_upstream': [
                'SSRCGCC',     # BREu (S=G/C, R=A/G)
                'GCGCCC',      # バリアント
            ],
            'BRE_downstream': [
                'RTDKKKK',     # BREd (R=A/G, D=A/G/T, K=G/T)
            ],
            
            # Motif Ten Element (MTE)
            'MTE': [
                'CSARCSSAACGS', # コンセンサス
            ],
            
            # X Core Promoter Element 1 (XCPE1)
            'XCPE1': [
                'VCPYV',       # バリアント複数
                'DSGYGGRASM',  # 別のパターン
            ],
            
            # CpG island promoter elements
            'CpG_island': [
                'CGCG',        # CpG dinucleotide
                'GCGC',        # リバース
                'CCGCGC',      # 長いバージョン
            ],
        }
    
    #Classからインスタンス化されたオブジェクトのメソッドを追加する。
    #このメソッドは、上位100個のプロモーターのリストをスライスして抽出。
    def get_tissue_specific_promoters(self, tissue_type='liver', top_n=100):
        """
        組織特異的なコアプロモーター要素を取得
        
        Parameters:
        -----------
        tissue_type : str
            対象組織タイプ (liver, brain, heart, immune, stem, muscle)
        top_n : int
            取得する上位数
        
        Returns:
        --------
        list
            コアプロモーター配列のリスト
        """
        # 全てのコアプロモーター要素を統合
        all_promoters = []
        for element_type, sequences in self.core_promoter_elements.items():
            all_promoters.extend(sequences)
        
        return all_promoters[:top_n]
    
    def get_transcription_factor_motifs(self, cell_type='liver', top_n=50):
        """
        組織特異的転写因子結合モチーフを取得
        
        Parameters:
        -----------
        cell_type : str
            細胞/組織タイプ
        top_n : int
            取得する数
        
        Returns:
        --------
        list
            TFBS配列のリスト
        """
        # 組織特異的転写因子を取得
        tissue_tfs = self.tissue_specific_tfs.get(cell_type, [])
        
        # それらのモチーフを取得
        motifs = []
        for tf in tissue_tfs:
            if tf in self.transcription_factor_motifs:
                motifs.append(self.transcription_factor_motifs[tf])
        
        # ユビキタス転写因子も追加（広範囲に重要）
        universal_tfs = ['SP1', 'AP1', 'E2F1', 'YY1', 'NRF1']
        for tf in universal_tfs:
            if tf in self.transcription_factor_motifs:
                motifs.append(self.transcription_factor_motifs[tf])
        
        return motifs[:top_n]
    
    def expand_degenerate_sequences(self, sequence):
        """
        縮重塩基コードを実際の配列に展開
        W = A/T, Y = C/T, R = A/G, N = A/T/G/C, など
        """
        degenerate_map = {
            'W': ['A', 'T'],
            'Y': ['C', 'T'],
            'R': ['A', 'G'],
            'N': ['A', 'T', 'G', 'C'],
            'S': ['G', 'C'],
            'K': ['G', 'T'],
            'M': ['A', 'C'],
            'V': ['A', 'C', 'G'],
            'H': ['A', 'C', 'T'],
            'D': ['A', 'G', 'T'],
            'B': ['C', 'G', 'T'],
        }
        
        # 縮重塩基を見つける
        for deg_base, alternatives in degenerate_map.items():
            if deg_base in sequence:
                expanded = []
                for alt in alternatives:
                    expanded.extend(
                        self.expand_degenerate_sequences(
                            sequence.replace(deg_base, alt, 1)
                        )
                    )
                return expanded
        return [sequence]
    
    def create_promoter_variants(self, core_sequence, n_variants=5):
        """
        コアプロモーター配列のバリアントを作成
        """
        variants = [core_sequence]
        
        # 1塩基置換バリアント
        bases = ['A', 'T', 'G', 'C']
        for i in range(min(len(core_sequence), 2)):  # 最初の2塩基のみ
            for base in bases:
                if base != core_sequence[i]:
                    variant = core_sequence[:i] + base + core_sequence[i+1:]
                    variants.append(variant)
                    if len(variants) >= n_variants:
                        return variants[:n_variants]
        
        return variants[:n_variants]
    
    def get_all_tissue_info(self):
        """
        全組織の転写因子情報を取得
        
        Returns:
        --------
        dict
            組織別の転写因子情報
        """
        info = {}
        for tissue in self.tissue_specific_tfs.keys():
            tfs = self.tissue_specific_tfs[tissue]
            motifs = [self.transcription_factor_motifs.get(tf, 'N/A') for tf in tfs]
            info[tissue] = {
                'transcription_factors': tfs,
                'count': len(tfs),
                'motifs': motifs
            }
        return info


def create_fantom5_based_library(tissue_type='liver', 
                                 cell_type='HepG2',
                                 n_cores=20,
                                 n_tfbs=15,
                                 include_variants=True,
                                 output_csv='fantom5_promoter_library.csv'):
    """
    FANTOM5データに基づいた人工プロモーターライブラリを作成
    
    Parameters:
    -----------
    tissue_type : str
        対象組織タイプ
    cell_type : str
        対象細胞タイプ
    n_cores : int
        使用するコアプロモーター数
    n_tfbs : int
        使用する転写因子結合部位数
    include_variants : bool
        配列バリアントを含めるか
    output_csv : str
        出力ファイル名
    """
    
    handler = FANTOM5DataHandler()
    
    # 1. FANTOM5から組織特異的コアプロモーターを取得
    print(f"\n=== ステップ1: {tissue_type}組織特異的プロモーターの取得 ===")
    core_promoters = handler.get_tissue_specific_promoters(tissue_type, n_cores)
    print(f"取得: {len(core_promoters)}個のコアプロモーター要素")
    
    # 2. 転写因子結合モチーフを取得
    print(f"\n=== ステップ2: {cell_type}細胞型のTFBSモチーフの取得 ===")
    tfbs_motifs = handler.get_transcription_factor_motifs(cell_type, n_tfbs)
    print(f"取得: {len(tfbs_motifs)}個のTFBSモチーフ")
    
    # 3. 縮重配列を展開
    print(f"\n=== ステップ3: 縮重配列の展開 ===")
    expanded_cores = []
    for core in core_promoters:
        expanded = handler.expand_degenerate_sequences(core)
        expanded_cores.extend(expanded[:3])  # 各コアから最大3バリアント
    
    expanded_tfbs = []
    for tfb in tfbs_motifs:
        expanded = handler.expand_degenerate_sequences(tfb)
        expanded_tfbs.extend(expanded[:2])  # 各TFBSから最大2バリアント
    
    print(f"展開後: {len(expanded_cores)}個のコアプロモーター")
    print(f"展開後: {len(expanded_tfbs)}個のTFBS")
    
    # 4. プロモーターライブラリ生成
    print(f"\n=== ステップ4: プロモーターライブラリの生成 ===")
    library_df = generate_promoter_library(
        core_list=expanded_cores,
        tfb_list=expanded_tfbs,
        promoter_types=['Coplanar', 'Opposite'],
        coplanar_spacer=10,
        opposite_spacer=5,
        output_csv=output_csv
    )
    
    # 5. 統計情報を追加
    library_df['Tissue_Type'] = tissue_type
    library_df['Cell_Type'] = cell_type
    library_df['GC_Content_Core'] = library_df['Core_Promoter'].apply(
        lambda x: (x.count('G') + x.count('C')) / len(x) * 100 if len(x) > 0 else 0
    )
    library_df['GC_Content_TFB'] = library_df['TFB'].apply(
        lambda x: (x.count('G') + x.count('C')) / len(x) * 100 if len(x) > 0 else 0
    )
    
    # 再保存
    if output_csv:
        library_df.to_csv(output_csv, index=False)
        print(f"\n最終ライブラリを{output_csv}に保存しました")
    
    print(f"\n=== 完成！ ===")
    print(f"総プロモーター数: {len(library_df):,}")
    print(f"ユニークなコア配列: {library_df['Core_Promoter'].nunique()}")
    print(f"ユニークなTFBS配列: {library_df['TFB'].nunique()}")
    
    return library_df


# ============================================================================
# 使用例
# ============================================================================

if __name__ == "__main__":
    
    print("=" * 70)
    print("FANTOM5ベース 人工プロモーターライブラリ作成システム（強化版）")
    print("=" * 70)
    
    # 例1: 肝臓特異的プロモーターライブラリ
    print("\n【肝臓特異的ライブラリ】")
    liver_library = create_fantom5_based_library(
        tissue_type='liver',
        cell_type='liver',
        n_cores=15,
        n_tfbs=12,
        output_csv='liver_promoter_library_enhanced.csv'
    )
    
    print("\n" + "=" * 70)
    print("ライブラリのサンプル:")
    print("=" * 70)
    print(liver_library.head(10))
    
    print("\n" + "=" * 70)
    print("配列長の統計:")
    print("=" * 70)
    if 'Coplanar_Length' in liver_library.columns:
        print(liver_library[['Coplanar_Length', 'Opposite_length']].describe())
    
    # 例2: 脳特異的プロモーターライブラリ
    print("\n\n【脳特異的ライブラリ】")
    brain_library = create_fantom5_based_library(
        tissue_type='brain',
        cell_type='brain',
        n_cores=15,
        n_tfbs=12,
        output_csv='brain_promoter_library_enhanced.csv'
    )
    
    print("\n脳特異的ライブラリ:")
    print(brain_library.head(5))

FANTOM5ベース 人工プロモーターライブラリ作成システム（強化版）

【肝臓特異的ライブラリ】

=== ステップ1: liver組織特異的プロモーターの取得 ===
取得: 15個のコアプロモーター要素

=== ステップ2: liver細胞型のTFBSモチーフの取得 ===
取得: 12個のTFBSモチーフ

=== ステップ3: 縮重配列の展開 ===
展開後: 33個のコアプロモーター
展開後: 15個のTFBS

=== ステップ4: プロモーターライブラリの生成 ===
ライブラリをliver_promoter_library_enhanced.csvに保存しました

最終ライブラリをliver_promoter_library_enhanced.csvに保存しました

=== 完成！ ===
総プロモーター数: 495
ユニークなコア配列: 29
ユニークなTFBS配列: 10

ライブラリのサンプル:
               ID Core_Promoter            TFB              Coplanar_sequence  \
0  Promoter_00001        TATAAA  GTTAATAATTAAC  GTTAATAATTAACNNNNNNNNNNTATAAA   
1  Promoter_00002        TATAAA  GTTAATTATTAAC  GTTAATTATTAACNNNNNNNNNNTATAAA   
2  Promoter_00003        TATAAA  GTTAATGATTAAC  GTTAATGATTAACNNNNNNNNNNTATAAA   
3  Promoter_00004        TATAAA         AGGTCA         AGGTCANNNNNNNNNNTATAAA   
4  Promoter_00005        TATAAA         AGGTCA         AGGTCANNNNNNNNNNTATAAA   
5  Promoter_00006        TATAAA         GGGTCA         GGGTCANNNNNNNNNNTATAAA   
6  Promoter_0000